<a href="https://colab.research.google.com/github/zy55164448/cdh2/blob/master/colab_katago_en_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document shows how to run KataGo on Colab, and how to connect it using `Sabaki`, `Lizzie` (or other GTP engine supported apps) in your local machine.

# Step 1: Run KataGo and SSH server on Colab Server

Before running, you should prepare these 3 values: `NGROK_TOKEN`, `USER_NAME` and `USER_PASSWORD`. 

* NGROK_TOKEN - You can register or login in `ngrok` website here: [https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken). Then you can find your own ngrok token.
* USER_NAME - You can use any name as you wish. (should only contains letters or digits)
* USER_PASSWORD - You can use any password as you wish. (should only contains letters or digits)

`USER_NAME` and `USER_PASSWORD` will be used in `Sabaki` or `Lizzie` as engine options. (More details will be elaborated in Step 2 below).

Change these 3 values of your own in the below code (the first 3 lines). Then click the `Run` button. 

It may take about 2~3 minutes to complete running the code. 

If you have any uncertainty for this step, you can also check this simple tutorial video: 
https://monosnap.com/file/gi2nIGtYXkosLInuDb9j5HQQkwB72w

In [ ]:
NGROK_TOKEN="1jLD11oR0t2zqCHWXO23rhngHss_4C8S1HxaQpDGa8WhCzvQR"
USER_NAME="zy55164448"
USER_PASSWORD="zhyong64"

# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="AUTO"
# supports: 40b, 30b, 20b, 40b-large or AUTO
WEIGHT_FILE="AUTO" 

import subprocess
if KATAGO_BACKEND == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND="CUDA"
  else:
    KATAGO_BACKEND="OPENCL"
!echo "Using Katago Backend: " $KATAGO_BACKEND

if WEIGHT_FILE == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaK80" or gpu_name == "TeslaP4":
    WEIGHT_FILE="20b"
  else:
    WEIGHT_FILE="40b"

!echo "Using Katago Weight: " $WEIGHT_FILE

weight_urls = {
    '40b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b40c256x2-s5095420928-d1229425124.bin.gz',
    '30b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz',
    '20b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170e-b20c256x2-s5303129600-d1228401921.bin.gz',
    '40b-large': 'https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/40b384.bin.gz'
}

# Install useful stuff
! apt-get update 1>/dev/null
! apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
! pip install oss2 1>/dev/null
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/ngrok -O ngrok
!chmod +x /content/ngrok
!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/

#download the weights
weight_url = weight_urls[WEIGHT_FILE]
!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
!rm -rf weight.bin.gz
!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

# SSH setting
! echo "root:$USER_PASSWORD" | chpasswd
! echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
! echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

! mkdir -p /root/.ssh

# generate the keys
#!ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa <<<y 2>&1 >/dev/null
#!cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys

! service ssh restart > /dev/null

# Run ngrok
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
! sleep 5

import oss2
import requests
import json
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
raw_ssh = r.json()['tunnels'][0]['public_url']
ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
ssh_option = {
    'host': ssh_args[0],
    'port': int(ssh_args[1]),
    'user': 'root'
}

ssh_option_json = json.dumps(ssh_option)

endpoint = 'http://oss-cn-beijing.aliyuncs.com'
auth = oss2.Auth('LTAI4G3tthX2R3Z8KY6Bdbs3', 'Q22UstMdKX8zZY9BqkGWWd2XbEBvPH')
bucket = oss2.Bucket(auth, endpoint, 'kata-config')

key = USER_NAME + '.ssh.json'
bucket.put_object(key, ssh_option_json)

!echo "done!"



Using Katago Backend:  CUDA
Using Katago Weight:  40b


W: Operation was interrupted before it could finish
/content
Cloning into 'katago-colab'...
remote: Enumerating objects: 102, done.
remote: Total 102 (delta 0), reused 0 (delta 0), pack-reused 102
Receiving objects: 100% (102/102), 32.42 KiB | 6.48 MiB/s, done.
Resolving deltas: 100% (46/46), done.


ConnectionError: ignored

#Step 2: Connect Colab KataGo via Sabaki or Lizzie

## 1) first, download a colab-katago client app
Here are the download Links:  
**For Windows Users (64bit windows)**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.windows.zip  
**For Linux Users**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.linux.zip  
**For Mac OSX Users**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.mac.zip  


**after download completed, unzip it, you can get a binary program naming  colab-katago or colab-katago.exe**

## 2) then, add engine in Sabaki or Lizzie

To configure the engine in `Sabaki` or `Lizzie`, you just need to fill the absolute path of your `colab-katago` program (which you've downloaded just now), and the username, password(i.e, the `USER_NAME`, `USER_PASSWORD` you configured in Step 1).


**Sabaki Example**:  

![Sabaki Example Image](
https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200725_12.png
)  

**Lizzie Example**:  

![Lizzie Example Image](https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200725_14.png
)

## More Config Options (Optional)
You can use the following engine options to limits the KataGo Search `visits` or `time` (in seconds) in genmove mode. For example: 

```
<AbsolutePathOfColabKataGoProglem> <USER_NAME> <USER_PASSWORD> 30s
```
The above options `30s` limits the search time for each move to 30 seconds.
```
<AbsolutePathOfColabKataGoProglem> <USER_NAME> <USER_PASSWORD> 1600v
```
The above options `1600v` limits the search visits for each move to 1600 visits.

You can change the numbers 30 or 1600 in the above to any number as you want.

#The following sections are for debugging only, you can ignore.

**Shows the Colab GPU Info**

In [ ]:
!nvidia-smi

Get your ssh login info
*ssh login account is root, the login password is the `USER_PASSWORD` you configured in the previous steps*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

ssh root@0.tcp.ngrok.io -p 12828
